In [3]:
import urllib.request
import os
import zipfile
import pandas as pd
from pandas import DataFrame
import geopandas as gpd
from shapely import wkt

In [4]:
mesh1_path = 'I:/マイドライブ/水道管劣化予測/水道管劣化予測/src/国土数値情報/土地利用メッシュ/mesh_for_work/mesh1_code/mesh1.csv'
mesh1_code = pd.read_csv(mesh1_path)

mesh1_code = pd.DataFrame(mesh1_code, columns=['Name'])

In [5]:
# 平成28年のデータの読み込み
base_url = "https://nlftp.mlit.go.jp/ksj/gml/data/G02/G02-22/G02-22_"
# mesh1_codeが前のRのコードの結果として得られるリストまたは配列と仮定します
# mesh1_codeの値を使ってURLを組み立て
url_list = [base_url + str(code) + "-jgd_GML.zip" for code in mesh1_code['Name']]

In [4]:
disable_mesh1_list = []

for i, url in enumerate(url_list):
    # 保存ファイル名の指定
    destfile = f"I:/マイドライブ/水道管劣化予測/水道管劣化予測/src/国土数値情報/平年値メッシュ/weather_mesh_for_work/weather_zip/weather_mesh_{mesh1_code['Name'][i]}.zip"
    
    # ダウンロード（データがない場合はスキップ）
    try:
        urllib.request.urlretrieve(url, destfile)
    except Exception as e:
        print(f"{url} did not work out due to {e}")
        disable_mesh1_list.append(url)
        
print(disable_mesh1_list)

https://nlftp.mlit.go.jp/ksj/gml/data/G02/G02-22/G02-22_3036-jgd_GML.zip did not work out due to HTTP Error 404: Not Found
https://nlftp.mlit.go.jp/ksj/gml/data/G02/G02-22/G02-22_4440-jgd_GML.zip did not work out due to HTTP Error 404: Not Found
https://nlftp.mlit.go.jp/ksj/gml/data/G02/G02-22/G02-22_4739-jgd_GML.zip did not work out due to HTTP Error 404: Not Found
https://nlftp.mlit.go.jp/ksj/gml/data/G02/G02-22/G02-22_5038-jgd_GML.zip did not work out due to HTTP Error 404: Not Found
https://nlftp.mlit.go.jp/ksj/gml/data/G02/G02-22/G02-22_6740-jgd_GML.zip did not work out due to HTTP Error 404: Not Found
['https://nlftp.mlit.go.jp/ksj/gml/data/G02/G02-22/G02-22_3036-jgd_GML.zip', 'https://nlftp.mlit.go.jp/ksj/gml/data/G02/G02-22/G02-22_4440-jgd_GML.zip', 'https://nlftp.mlit.go.jp/ksj/gml/data/G02/G02-22/G02-22_4739-jgd_GML.zip', 'https://nlftp.mlit.go.jp/ksj/gml/data/G02/G02-22/G02-22_5038-jgd_GML.zip', 'https://nlftp.mlit.go.jp/ksj/gml/data/G02/G02-22/G02-22_6740-jgd_GML.zip']


In [5]:
# zipファイルパスの取得
zip_path = 'I:/マイドライブ/水道管劣化予測/水道管劣化予測/src/国土数値情報/平年値メッシュ/weather_mesh_for_work/weather_zip'
ziplist = [os.path.join(zip_path, f) for f in os.listdir(zip_path) if f.endswith('.zip')]

# 保存ファイル名用
mesh_file = 'I:/マイドライブ/水道管劣化予測/水道管劣化予測/src/国土数値情報/平年値メッシュ/weather_mesh_for_work/weather_extract'
filename = [f for f in os.listdir(mesh_file) if f.endswith('.zip')]

# zipファイルの解凍
for i, zip_path in enumerate(ziplist):
    # zipファイルの解凍
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(mesh_file)

In [6]:
# shpファイルのフルパスを作成
root_path = "I:/マイドライブ/水道管劣化予測/水道管劣化予測/src/国土数値情報/平年値メッシュ/weather_mesh_for_work"
shp_list = []
for dirpath, dirnames, filenames in os.walk(root_path):
    for filename in filenames:
        if filename.endswith('.shp'):
            shp_list.append(os.path.join(dirpath, filename))

# shpファイルの読み込み
dataframes = []
for shp_path in shp_list:
    dataframes.append(gpd.read_file(shp_path, encoding='shift-jis'))

In [7]:
# マージ
national_1km_weather = gpd.GeoDataFrame(columns=[dataframes[0].columns])

disable_mesh_gdf = []
for gdf in dataframes:
    #座標系の変換
    dst_proj = 4326  # 変換後の座標系を指定
    gdf = gdf.to_crs(epsg=dst_proj)  # 変換後座標に変換
    if set(dataframes[0]).issubset(gdf.columns):
        national_1km_weather = pd.concat([national_1km_weather, gdf], ignore_index=True, verify_integrity=True)
        
    else:
        print(f"{gdf} does not have the required columns and will not be merged.")
        disable_mesh_gdf.append(gdf)

: 

In [63]:
pref_polygon_path = 'I:/マイドライブ/水道管劣化予測/水道管劣化予測/src/国土数値情報/土地利用メッシュ/mesh_for_work/prefecture_polygon/prefecture.geojson'
pref_polygon = gpd.read_file(pref_polygon_path)

i = 1

for index, pref in pref_polygon.iloc[2:].iterrows():
    
    # GeoDataFrameを作成し、'geometry'カラムを設定します
    gdf_pref = gpd.GeoDataFrame(geometry=[pref['geometry']])
    
    if gdf_pref.crs is None:
	    gdf_pref.set_crs(epsg=4326, inplace=True)
    
    pref_100m_mesh = gpd.sjoin(gdf_pref, national_100m_mesh, op='intersects')
    pref_100m_mesh['pref'] = pref[0]
    pref_100m_mesh['name'] = pref[1]
    
    pref_100m_mesh = pref_100m_mesh[['pref', 'name', 'メッシュ', '土地利用種', '撮影年月日']]

    pref_100m_mesh = pd.merge(pref_100m_mesh, national_100m_mesh[['メッシュ', 'geometry']], on='メッシュ')
    
    print(f'--{i}県の出力中--')
    pref_100m_path = f'I:/マイドライブ/水道管劣化予測/水道管劣化予測/src/国土数値情報/平年値メッシュ/平年値メッシュ_全国版/{i}_1kM_weather.geojson'
    gpd.GeoDataFrame(pref_100m_mesh, geometry='geometry').to_file(pref_100m_path)
    print(f'--{i}県の出力完了--')
    
    i += 1

c:\Users\imai_14\anaconda3\envs\microbase\lib\site-packages\IPython\core\interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


--3県の出力中--
--3県の出力完了--


c:\Users\imai_14\anaconda3\envs\microbase\lib\site-packages\IPython\core\interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


--4県の出力中--
--4県の出力完了--


c:\Users\imai_14\anaconda3\envs\microbase\lib\site-packages\IPython\core\interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


...

c:\Users\imai_14\anaconda3\envs\microbase\lib\site-packages\IPython\core\interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


--46県の出力中--
--46県の出力完了--


c:\Users\imai_14\anaconda3\envs\microbase\lib\site-packages\IPython\core\interactiveshell.py:3338: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


--47県の出力中--
--47県の出力完了--


: 